In [2]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import pymongo
from pymongo import MongoClient

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
df = pd.read_csv('final_unemp.csv')
df

In [ ]:
df.dtypes

In [ ]:
df.drop(columns='Metropolitan_Area', inplace=True)
df.head()

In [ ]:
# for loop to predict 2024 values
predictions = []
for i in range(0,383):
    y = df.iloc[i, 1:].values
    series = pd.Series(y, dtype='float')
    model = ARIMA(series, order=(1, 1, 0))
    model_fit = model.fit()
    pred = model_fit.forecast(5)
    predictions.append(pred.values.tolist()[-1])
df['2024_Unem_ROC'] = predictions

In [ ]:
df.head()

In [14]:
df.sort_values(by='2024_Unem_ROC')

,CBSA,2015,2016,2017,2018,2019,2024_Unem_ROC
33,12940,5.5,5.0,4.5,4.1,3.7,1.748249
53,22020,2.5,2.5,2.3,2.0,1.9,1.777604
382,49180,5.4,5.0,4.5,4.1,3.7,1.820066
241,45060,5.4,5.0,4.5,4.1,3.7,1.820066
149,19340,5.6,5.1,4.6,4.2,3.8,1.848223
...,...,...,...,...,...,...,...
168,12100,9.6,8.7,7.5,6.5,6.8,7.247898
351,32900,11.4,10.6,9.4,8.4,8.1,7.255012
337,47300,11.6,11.2,10.4,9.8,9.6,9.136842
191,49740,21.7,18.8,16.9,16.7,16.4,15.551137


In [7]:
df_pred = df[['CBSA', '2024_Unem_ROC']].copy()
df_pred

,CBSA,2024_Unem_ROC
0,14020,2.293605
1,10420,2.717638
2,14860,1.848223
3,16060,3.372314
4,19740,2.700000
...,...,...
378,43340,4.682347
379,46060,2.693683
380,46660,3.193397
381,47380,2.087679


In [8]:
df_pred['Unem_Score'] = df_pred['2024_Unem_ROC'].rank(ascending=False)

In [9]:
df_pred

,CBSA,2024_Unem_ROC,Unem_Score
0,14020,2.293605,313.5
1,10420,2.717638,182.5
2,14860,1.848223,378.5
3,16060,3.372314,117.5
4,19740,2.700000,185.5
...,...,...,...
378,43340,4.682347,48.5
379,46060,2.693683,192.5
380,46660,3.193397,127.5
381,47380,2.087679,348.5


In [10]:
# Connection to mongo
client = MongoClient('mongodb+srv://<username>:<password>@cluster0.l3pqt.mongodb.net/MSA?retryWrites=true&w=majority')
# Select database
db = client['MSA']

In [11]:
# create new collection in mongo
arima_unem_pred_score = db.arima_unem_pred_score

In [12]:
df_dict = df_pred.to_dict(orient='records')

In [13]:
arima_unem_pred_score.insert_many(df_dict)